Подключаем библиотеки

In [ ]:

from flask import Flask, request, Response
from nornir import InitNornir
import pynetbox
import ipaddress
from jinja2 import Environment, FileSystemLoader
import json
import re

Инициализируем nornir

In [ ]:
# Init Nornir
nr = InitNornir(
    inventory={
        "plugin": "NetBoxInventory2",
        "options": {
            "nb_url": "http://10.30.1.101:8000",
            "nb_token": "dc5b766afda60e647a9fcc384c736a9689da8618",
            "group_file": "./inventory/groups.yml",
            "defaults_file": "./inventory/defaults.yml",
        },
    },
)

Инициализируем pynetbox

In [ ]:
# Init pynetbox
nb_url = nr.config.inventory.options['nb_url']
nb_token = nr.config.inventory.options['nb_token']
nb = pynetbox.api(
    nb_url,
    token=nb_token
)

In [ ]:
def configure_interface_ipv4_address(netbox_ip_address):
    """
    Извлечение IPv4 адреса, маски, сети, префикса, шлюза.
    :param netbox_ip_address: IPv4 адрес (IP/Prefix)
    :return: ip4_address
    """

    ip4_address = format(ipaddress.IPv4Interface(netbox_ip_address).ip)
    ip4_netmask = format(ipaddress.IPv4Interface(netbox_ip_address).netmask)
    ip4_network = format(ipaddress.IPv4Interface(netbox_ip_address).network)
    ip4_prefix = format(ipaddress.IPv4Network(ip4_network).prefixlen)
    ip4_broadcast = format(ipaddress.IPv4Network(ip4_network).broadcast_address)
    ip4_gateway = format(list(ipaddress.IPv4Network(ip4_network).hosts())[-1])
    #print(ip4_address,ip4_netmask, ip4_network, ip4_prefix, ip4_broadcast, ip4_gateway)

    return(ip4_address)

In [ ]:
def test_for_equal(device_name):
    """
    Проверяем файл с конфигурацией на наличие в нем каких-либо настроек, связанных с передаваемым параметром
    :param device_name
    :return: None 
    """

    start,end = -2,-2 # start и end должны быть не пустыми и меньше -1
    name = device_name.strip().casefold() # избавляемся от пробелов и игнорируем регистр
    with open("result.conf", 'r') as f: # открываем файл для чтения
        config = f.readlines() # считывем построчно и получаем список
    for line in config:
        if line.startswith('host'): # если в начале строки попадается 'host'
            if name in (s for s in line.strip().casefold().split()): # перебираем строку поэлементно на совпадение с 'name'
                start = config.index(line) # запоминаем индекс
                for i in range(start+1,len(config)): # продолжаем перебирать строки, начиная с индекса
                    if '}' in (s for s in list(config[i])): # разбиваем строку на отдельные символы и проверяем на совпадение с '}'
                        end = i
                        break
    
    if start >= 0 and end >=start : # значения для среза обязательно должны быть, и второе (end) должно быть не меньше первого (start)
        del config[start:end+1] # делаем срез списка (избавляемся от строк)
        with open("result.conf", 'w') as w:
            w.writelines(config)
            w.close()
        print("'{}' is find and deleted there config...".format(device_name))
    else: print("Configuration file does not contain '{}'...".format(device_name))
    print(config)
    

#test_for_equal('SRV-PEV-5')

In [ ]:
def dhcpd_config_file(j2_device_name,j2_mac_address,j2_ip_address,event='None'):
    """ 
    Заполнение шаблона значениями
    :param j2_device_name: устройство 
    :param j2_mac_address: mac адрес
    :param j2_ip_address: ip адрес
    :return: None
    """
    
    test_for_equal(j2_device_name)
    
    print(event)
    if event != 'delete':
        templates_path = "./templates/"
        environment = Environment(loader=FileSystemLoader(templates_path))
        template = environment.get_template("dhcpd_static.template")

        content = template.render(
        device_name = j2_device_name,
        mac_address = j2_mac_address,
        ip_address = j2_ip_address
        )
        
        print(content)

        # Сохраним получившийся текст
        with open('result.conf', 'a') as fp: 
            fp.write(content + '\n')
            fp.close()

        # Прочитаем записанный файл
        with open('result.conf', 'r') as fp: 
            print(fp.read())

#dhcpd_config_file('SRV-PEV-1','None', '1.1.1.1', 'create')

In [ ]:
def delete_ip_address(netbox_interface,netbox_ip_address,netbox_address_family):
    """  
    Удаление ip адреса
    :param netbox_interface: ссылка на объект интерфейса pynetbox
    :param netbox_ip_address: IPv4 адрес (IP/Prefix)
    :param netbox_address_family: Версия IP (4|6)
    :return: None
    """
    
    print(f"Removing address {netbox_ip_address} "
          f"from interface '{netbox_interface.name}' "
          f"on device '{netbox_interface.device.name}'...")

    if netbox_address_family == 4:
        ip_address = configure_interface_ipv4_address(netbox_ip_address)
        device_name = netbox_interface.device.name
        mac_address = netbox_interface.mac_address
    else:
        print("IPv6")
        #configure_interface_ipv6_address(netbox_ip_address)
    dhcpd_config_file(device_name,mac_address,ip_address,event='delete')

In [ ]:
def create_ip_address(netbox_interface,netbox_ip_address,netbox_address_family):
    """  
    Создание ip адреса
    :param netbox_interface: ссылка на объект интерфейса pynetbox
    :param netbox_ip_address: IPv4 адрес (IP/Prefix)
    :param netbox_address_family: Версия IP (4|6)
    :return: None
    """

    print(f"Assigning address {netbox_ip_address} "
          f"to interface '{netbox_interface.name}' "
          f"on device '{netbox_interface.device.name}'...")
    
    if netbox_address_family == 4:
        ip_address = configure_interface_ipv4_address(netbox_ip_address)
        device_name = netbox_interface.device.name
        mac_address = netbox_interface.mac_address
    else:
        print("IPv6")
        #configure_interface_ipv6_address(netbox_ip_address)
    
    dhcpd_config_file(device_name,mac_address,ip_address,event='create')

In [ ]:
def update_ip_address(netbox_interface,snapshot_json,netbox_ip_address,netbox_address_family):
    """  
    Изменение ip адреса
    :param netbox_interface: ссылка на объект интерфейса pynetbox
    :param netbox_ip_address: IPv4 адрес (IP/Prefix)
    :param netbox_address_family: Версия IP (4|6)
    :return: None
    """

    print("Updating IP address...")
    #print(snapshot_json)
    if snapshot_json:
        try:
            old_interface_id = snapshot_json["prechange"]["assigned_object_id"]

            if old_interface_id != netbox_interface.id:
                # Старое назначение принадлежит другому интерфейсу. 
                # Изменение конфигурации перед настройкой нового устройства.
                old_interface_data = nb.dcim.interfaces.get(old_interface_id)
                if not old_interface_data.mgmt_only:
                    delete_ip_address(  netbox_interface,
                                        netbox_ip_address,
                                        netbox_address_family)
        except AttributeError:
            print("Address not previously assigned")
        except ValueError:
            print("Address not previously assigned")

    if netbox_address_family == 4:
        ip_address = configure_interface_ipv4_address(netbox_ip_address)
        device_name = netbox_interface.device.name
        mac_address = netbox_interface.mac_address
    else:
        print("IPv6")
        #configure_interface_ipv6_address(netbox_ip_address)
    
    dhcpd_config_file(device_name,mac_address,ip_address,event='update')

Создаем функцию для манипулирования IP адресами

In [ ]:
def manage_interface_ip_address():
        
    #get_device = request.json["data"]["assigned_object"]["device"]["name"]
    get_device_interface = nb.dcim.interfaces.get(request.json["data"]["assigned_object_id"])    
    get_device_ips = request.json["data"]["address"]
    get_address_family = request.json["data"]["family"]["value"]

    if get_device_interface.mgmt_only: # проверяем, является ли интерфейс management интерфейсов
        print("\tManagement interface, no changes will be performed...")
    else:
            if request.json["event"] == "deleted": # IP адрес будет удален

                delete_ip_address(  netbox_interface=get_device_interface,
                                    netbox_ip_address=get_device_ips,
                                    netbox_address_family=get_address_family
                                    )

            elif request.json["event"] == "created": # IP адрес будет добавлен.

                create_ip_address(netbox_interface=get_device_interface,
                                    netbox_ip_address=get_device_ips,
                                    netbox_address_family=get_address_family)

            elif request.json["event"] == "updated":
                # Details of the IP have changed.  Determine if it was previously
                # assigned to a different device / interface. If so, unconfigure
                # that interface first and configure on the new one.
                update_ip_address(  netbox_interface=get_device_interface,
                                    snapshot_json=request.json.get("snapshots"),
                                    netbox_ip_address=get_device_ips,
                                    netbox_address_family=get_address_family
                                    )
    
    return Response(status=204)

Создаем экземпляр Flask

In [ ]:
# Create a Flask instance
app = Flask(__name__)
app.add_url_rule("/api/fixed_ip",
                 methods=["POST"],
                 view_func=manage_interface_ip_address)

if __name__ == "__main__":
    # If this script is called from the command line, instruct Flask to enable
    # debugging for the app and listen on every IP address on the specified
    # port.
    #app.debug = True
    app.run(host="0.0.0.0", port=8080)